<h3> R snippet: </h3>

- remove duplicates based on lat,long,species,year 
- add seasons info

Use R kernel to run

In [ ]:
library(tidyverse)

total = read.csv("GBif_Original.csv", stringsAsFactors = FALSE, sep="\t")

unq_rows = as.numeric(rownames(unique(total[c("species","year","decimalLatitude","decimalLongitude")])))
total_unq = total[unq_rows,]

total$coor = paste0(total$decimalLatitude, total$decimalLongitude)
total_unq$coor = paste0(total_unq$decimalLatitude, total_unq$decimalLongitude)
month_count = total %>% group_by(species, year, coor) %>% summarise(paste(unique(month), collapse = ", "))
colnames(month_count)[ncol(month_count)] = "months"

total_unq = total_unq[order(total_unq$species, total_unq$year, total_unq$coor),]
month_count = month_count[order(month_count$species, month_count$year, month_count$coor),]
#total_unq$months = month_count$`paste(unique(month), collapse = ", ")`
#sep_months = sapply(total_unq$months, strsplit, ", ")
sep_months = month_count$months

total_unq$Winter = 0
total_unq$Spring = 0
total_unq$Summer = 0
total_unq$Fall = 0
for (row in 1:nrow(total_unq)) {
  if (any(c(1,2,12) %in% sep_months[row][[1]][1])) {total_unq$Winter[row] = 1}
  if (any(c(3:5) %in% sep_months[row][[1]][1])) {total_unq$Spring[row] = 1}
  if (any(c(6:8) %in% sep_months[row][[1]][1])) {total_unq$Summer[row] = 1}
  if (any(c(9:11) %in% sep_months[row][[1]][1])) {total_unq$Fall[row] = 1}
  #if (row %% 15000 == 0) {print(paste("Loop is ", round(row/nrow(total_unq)*100), "% done", sep = ""))}
}

total_unq$coor = NULL

write.csv(total_unq, "GBif_R.csv")


<h3> Python snippets: </h3>

- convert dataset keys to dataset names
- add redList designation
- add common names from ITIS
- drop unnecessary columns
- add geometry from lat, long
- convert data to species per year in geojson

Use Python2 kernel to run

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# from geojson import GeometryCollection

import requests
import json
import pickle
import numpy as np

In [2]:
# Point data to geoJSON
import random
import sqlite3
import os

df_red = pd.read_csv("redlist_assessments.csv")
red_sciNames = df_red.scientificName

def some(x, n):
    return x.loc[random.sample(x.index, n)]

def dataSetNamesFromKey(df):
    k = df['datasetKey'].unique()
#     n = ([])
#     for key in k:
#         url = "http://api.gbif.org/v1/dataset/" + key
#         name = requests.get(url).json()['title']
#         n.append(name)
#     conv = dict(zip(k,n))  
    
#     pickle_out = open("dataSources.pickle",'wb') 
#     pickle.dump(conv, pickle_out)   
#     pickle_out.close()
    conv = pd.read_pickle("datasetKeyNames.pickle")
    
    df['datasetName'] = df['datasetKey']
    df = df.replace({'datasetName': conv})
    df = df.drop(columns='datasetKey')
    return df

# def observationsCountPerYear(df):
#     gb = df.groupby(['year'])
#     geo_df = df.dropna(subset = ['year'])
#     geo_df['year'].astype(int)

      # gbl2 missing
#     year_dict = gbl2.to_dict()
#     js = json.dumps(year_dict)
    
#     file = open('years_json.txt', 'w')
#     json.dump(year_dict, file)
#     file.close()

def addCommonNames(geo_df):
    
    # df_commons['common'] = df_commons[['language', 'vernacular_name']].apply(lambda x: ':'.join(x), axis=1)
    # df_commons = df_commons.groupby(['tsn'])['common'].apply(', '.join).reset_index()
    db = sqlite3.connect('ITIS.sqlite')
    df_commons = pd.read_sql_query("SELECT * from vernaculars", db)
    df_species = pd.read_sql_query("SELECT * from longnames", db)

    df_commons = df_commons[df_commons['language'] == "English"]
    df_commons = df_commons.rename({'vernacular_name': 'common'}, axis=1)
    df_commons = df_commons[['tsn', 'common']]
    
    df_species = df_species[df_species['completename'].isin(geo_df.species)]
    df_commons = df_commons[df_commons['tsn'].isin(df_species.tsn)]
    
    df_commons = df_commons.groupby(['tsn'])['common'].apply(', '.join).reset_index()
    df_species = df_species[df_species['tsn'].isin(df_commons.tsn)]

    df_common_species = df_species.merge(df_commons, on="tsn")
    df_common_species = df_common_species.rename({'completename': 'species'}, axis=1) 
    df_common_species = df_common_species.drop(['tsn'], 1)

    species = set(geo_df.species.unique())
    species_in_itis = set(df_common_species.species.unique())
    diff = list(species - species_in_itis)
    diff = {df_common_species.columns[0]: diff, df_common_species.columns[1]: 'Common Name unknown'}
    df = pd.DataFrame.from_dict(diff)
    df_species = pd.concat([df_common_species, df], sort = False)

    geo_df = geo_df.merge(df_species, on="species")
    return geo_df

def redList(x):
    if (x['species'] in red_sciNames.values):
        return df_red.loc[red_sciNames[red_sciNames == x['species']].index[0]].redlistCategory
    else:
        return float('nan')
    
    
def testingFilter(df):
    df = df[(df['year'] == 2017) | (df['year'] == 2016) | (df['year'] == 2015)]
    df = df[(df["species"] == "Corvus caurinus") | (df["species"] == "Turdus migratorius") |
           (df["species"] == "Larus glaucescens")]
    return df


In [3]:
def pointCSVtoJSONComplete(filename, num_rows=0):
    ##open dataframe from R script that removes duplicates based on lat, long, year & species
    #and adds seasons
    df = pd.read_csv(filename, low_memory=False)
    #drop nan species
    df = df.dropna(axis=0, subset=['species'])
    #keep num_rows if num_rows > 0
    if num_rows:
        df = some(df, num_rows)
    #convert dataset keys to dataset names    
    df = dataSetNamesFromKey(df)
    print("Converted dataset keys to names!")
    #add redList designation
    df['redList'] = df.apply(lambda x: redList(x), 1).values
    print("Added redlist designation!")
    #add common names from ITS
    df = addCommonNames(df)
    print("Added common names!")
    df = df[['species', 'Winter', 'Spring', 'Summer', 'Fall', 'datasetName', 'common', 
                     'redList', 'decimalLatitude', 'decimalLongitude', 'year']]
    #add geometry from lat, long
#     geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
#     #fix coordinate system
#     geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
#     geo_df = geo_df.drop(['decimalLatitude', 'decimalLongitude'], 1)
    #split into species per year as gejsons            
#     gb_year = geo_df.groupby(['year'])
#     for k_year, gp_year in gb_year:
#         gp_year = gp_year.drop(['year'], 1)
#         gp_species = gp_year.groupby(['species'])
#         os.mkdir('leaflet/gbif_year_species/' + str(int(k_year)))
#         for k_species, gp_species in gp_species:
#             gp_species.to_file("map_django/biodivmap/static/biodivmap/gbif_year_species" + str(int(k_year))
#                                + '/'+ k_species + ".geojson", driver="GeoJSON")
#         print(str(int(k_year)) + " done!")
    df.to_csv("Gbif Lesley.csv", encoding='latin1')  
    return df


In [5]:
BASE_PATH = 'map_django/biodivmap/static/biodivmap/'
taxonLevels = ['gbif','kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
file_count = 0

def pointCSVtoJSONPerYearPerTaxonLimited(filename, num_rows=0, testing_filter=False, lim=1000):
    ##open dataframe from R script that removes duplicates based on lat, long, year & species
    #and adds seasons
    df = pd.read_csv(filename, low_memory=False)
    #drop nan species
    df = df.dropna(axis=0, subset=['species'])
    #keep num_rows if num_rows > 0
    if num_rows:
        df = some(df, num_rows)
    if testing_filter:
        df = testingFilter(df)
    #convert dataset keys to dataset names    
    df = dataSetNamesFromKey(df)
    print("Converted dataset keys to names!")
    #add redList designation
    df['redList'] = df.apply(lambda x: redList(x), 1).values
    print("Added redlist designation!")
    #add common names from ITS
    df = addCommonNames(df)
    print("Added common names!")
    df = df[['Winter', 'Spring', 'Summer', 'Fall', 'datasetName', 'common', 
                     'redList', 'decimalLatitude', 'decimalLongitude', 'year',
            'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']]
    #add geometry from lat, long
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    #fix coordinate system
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    geo_df = geo_df.drop(['decimalLatitude', 'decimalLongitude'], 1)
    #split into species per year as gejsons            
    gb_year = geo_df.groupby(['year'])
    os.mkdir(BASE_PATH + 'gbif/')
    master_count = 0
    dbCon = sqlite3.connect("map_django/db.sqlite3")
    cur   = dbCon.cursor()
    for k_year, gp_year in gb_year:
        gp_year = gp_year.drop(['year'], 1)
        year_path = BASE_PATH + 'gbif/' + str(int(k_year)) + "/"
        os.mkdir(year_path)
        master_count += perTaxonWriter(0, gp_year, year_path, lim, cur, str(int(k_year)))
        print(str(int(k_year)) + " done!")
    dbCon.commit()
    dbCon.close()
    return geo_df, master_count

def perTaxonWriter(taxonIndex, df, path, lim, cur, year):
    if (taxonLevels[taxonIndex] == "species" or df.shape[0] <= lim):
        for spec in list(df.species.unique()):
            cur.execute("UPDATE species_year SET (%s) = ?  WHERE species = ?" %("year_"+ year),(path, spec,))
        df.to_file(path + "points"+ ".geojson", driver="GeoJSON")
        return 1
    else:
        gb_nextTaxonLevel = df.groupby([taxonLevels[taxonIndex+1]])
        inner_count = 0
        for k_nextTaxon, gp_nextTaxon in gb_nextTaxonLevel:
            next_path = path + k_nextTaxon + "/"
            os.mkdir(next_path)
            inner_count+=perTaxonWriter(taxonIndex+1, gp_nextTaxon, next_path, lim, cur,year)
        return inner_count
    

In [6]:
geo_df, count = pointCSVtoJSONPerYearPerTaxonLimited("GBif_R.csv")

Converted dataset keys to names!
Added redlist designation!
Added common names!


/Users/raghav/anaconda2/envs/jupy2/lib/python2.7/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


1700 done!
1800 done!
1818 done!
1856 done!
1857 done!
1859 done!
1860 done!
1872 done!
1874 done!
1875 done!
1876 done!
1877 done!
1878 done!
1879 done!
1880 done!
1881 done!
1882 done!
1883 done!
1884 done!
1885 done!
1886 done!
1887 done!
1888 done!
1889 done!
1890 done!
1891 done!
1892 done!
1893 done!
1894 done!
1895 done!
1896 done!
1897 done!
1898 done!
1899 done!
1900 done!
1901 done!
1902 done!
1903 done!
1904 done!
1905 done!
1906 done!
1907 done!
1908 done!
1909 done!
1910 done!
1911 done!
1912 done!
1913 done!
1914 done!
1915 done!
1916 done!
1917 done!
1918 done!
1919 done!
1920 done!
1921 done!
1922 done!
1923 done!
1924 done!
1925 done!
1926 done!
1927 done!
1928 done!
1929 done!
1930 done!
1931 done!
1932 done!
1933 done!
1934 done!
1935 done!
1936 done!
1937 done!
1938 done!
1939 done!
1940 done!
1941 done!
1942 done!
1943 done!
1944 done!
1945 done!
1946 done!
1947 done!
1948 done!
1949 done!
1950 done!
1951 done!
1952 done!
1953 done!
1954 done!
1955 done!
1956 done!

In [7]:
count

3406

In [ ]:
sum = 0
for key,gp in geo_df.groupby(["year"]):
    sum+=gp.species.unique().shape[0]
print(sum)

In [ ]:
def pointCSVtoJSONPerYearPerSpecies(filename, num_rows=0, testing_filter=False):
    ##open dataframe from R script that removes duplicates based on lat, long, year & species
    #and adds seasons
    df = pd.read_csv(filename, low_memory=False)
    #drop nan species
    df = df.dropna(axis=0, subset=['species'])
    #keep num_rows if num_rows > 0
    if num_rows:
        df = some(df, num_rows)
    if testing_filter:
        df = testingFilter(df)
    #convert dataset keys to dataset names    
    df = dataSetNamesFromKey(df)
    print("Converted dataset keys to names!")
    #add redList designation
    df['redList'] = df.apply(lambda x: redList(x), 1).values
    print("Added redlist designation!")
    #add common names from ITIS
    df = addCommonNames(df)
    print("Added common names!")
    df = df[['species', 'Winter', 'Spring', 'Summer', 'Fall', 'datasetName', 'common', 
                     'redList', 'decimalLatitude', 'decimalLongitude', 'year']]
    #add geometry from lat, long
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    #fix coordinate system
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    geo_df = geo_df.drop(['decimalLatitude', 'decimalLongitude'], 1)
    #split into species per year as gejsons            
    gb_year = geo_df.groupby(['year'])
    for k_year, gp_year in gb_year:
        gp_year = gp_year.drop(['year'], 1)
        gp_species = gp_year.groupby(['species'])
        os.mkdir('leaflet/gbif_year_species/' + str(int(k_year)))
        for k_species, gp_species in gp_species:
            gp_species.to_file("map_django/biodivmap/static/biodivmap/gbif_year_species" + str(int(k_year))
                               + '/'+ k_species + ".geojson", driver="GeoJSON")
        print(str(int(k_year)) + " done!")
        
    return geo_df


In [ ]:
geo_df = pointCSVtoJSONPerYearPerSpecies("Gbif_R.csv", testing_filter=True)

In [ ]:
df = pd.read_csv("Gbif_R.csv", low_memory=False)
df = df.dropna(subset = ['year'])
df.year = df['year'].astype(int)

In [ ]:
gp_series = df.groupby(['year']).size()

In [ ]:
year_counts = list(zip(gp_series.index, gp_series))

In [ ]:
years = list(gp_series.index)

In [ ]:
dbCon = sqlite3.connect("map_django/db.sqlite3")

In [ ]:
cur   = dbCon.cursor()

In [ ]:
#INSERT SPECIES
for spec in list(geo_df.species.unique()):
    cur.execute("INSERT INTO species_year (species) VALUES (?)",(spec,))
dbCon.commit()


In [ ]:
dbCon.close()

In [ ]:
cur.execute("UPDATE species_year SET (%s) = ?  WHERE species = ?" %("year_1700"),("path/", "Abagrotis baueri",))


In [ ]:
col_query = """CREATE TABLE species_year (
    species TEXT NOT NULL PRIMARY KEY,"""

for year in years:
    if (year == 2019):
        col_query+="""year_""" + str(year) + """ INTEGER DEFAULT 0"""
    else:
        col_query+="""year_""" + str(year) + """ INTEGER DEFAULT 0,"""
    
col_query += """);"""    

In [ ]:
cur.execute(col_query)

In [ ]:
cur.close()
dbCon.close()

In [ ]:
with open('map_django/years.pkl', 'wb') as f:
    pickle.dump(years, f)

In [ ]:
def pointCSVtoJSON(filename, num_rows=0):
    df = pd.read_csv(filename, delimiter="\t", low_memory=False)
    if (num_rows):
        df = some(df, num_rows)
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    
    # gbif_geo_df = gbif_geo_df[gbif_geo_df['species'].isin(red_sciNames)]
#     geo_df['redList'] = geo_df.apply(lambda x: int(x['species']in red_sciNames.values), 1).values
    geo_df['redList'] = geo_df.apply(lambda x: redList(x, red_sciNames), 1).values
    
    
    # Common Names ITIS
    # # Get names of all tables
    # c = conn.cursor()
    # c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # print(c.fetchall())
    
    geo_df = addCommonNames(geo_df)
    geo_df.to_file("leaflet/" + filename.split('.')[0] + '.geojson', driver="GeoJSON")
    
    return geo_df

def pointCSVtoJSONPerYear(filename, num_rows=0):
    df = pd.read_csv(filename, low_memory=False)
    if (num_rows):
        df = some(df, num_rows)
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    
    # gbif_geo_df = gbif_geo_df[gbif_geo_df['species'].isin(red_sciNames)]
#     geo_df['redList'] = geo_df.apply(lambda x: int(x['species']in red_sciNames.values), 1).values
    geo_df['redList'] = geo_df.apply(lambda x: redList(x, red_sciNames), 1).values
    
    
    # Common Names ITIS
    # # Get names of all tables
    # c = conn.cursor()
    # c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # print(c.fetchall())
    
    geo_df = addCommonNames(geo_df)
    
    geo_df = geo_df.drop(['decimalLatitude', 'decimalLongitude'], 1)
    
    gb = geo_df.groupby(['year'])

    for k, gp in gb:
#        gp.drop(['year'], 1)
       gp.to_file("leaflet/gbif/" + str(int(k)) + ".geojson", driver="GeoJSON")
    return geo_df


In [ ]:
geo_df = pointCSVtoJSONPerYear('GBif Trim June19.csv')

In [ ]:
df = pd.read_csv("GBif Trim June19.csv")

In [ ]:
df

In [ ]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
# str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = gpd.read_file("MVSEI2014/MVSEI2014.shp")

str_map = fix_crs(str_map)

In [ ]:
# # GeoJSON does not support multipolygon. Doesn't work
# str_map.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")

# # fiona doesn't work
# import fiona
# import json

# with fiona.open('ecological_reserves/BC_Eco_Reserves.shp') as source:
#     records = list(source)
# geo_json = {"type": "FeatureCollection","features": records}
# with open('leaflet/UBC_poly.geojson', 'w') as fp:
#     json.dump(geo_json, fp)

In [ ]:
# Convert multipolygon to single polygons

In [ ]:
geom_series = str_map.geometry

In [ ]:
def geom_apply(x):
    try:
        return list(x)
    except TypeError:
        return [x]

In [ ]:
geom_series = geom_series.apply(geom_apply)

In [ ]:
geom_series = geom_series.apply(pd.Series).stack()

In [ ]:
# # Quick but loses properties
# list_poly = list(geom_series)
# geoms = GeometryCollection(list_poly)
# geo_file = geojson.dumps(geoms)
# with open("leaflet/UBC_poly.geojson", "w") as text_file:
#     text_file.write(geo_file)

In [ ]:
# Careful! Deep copy required here to avoid chaining

df = pd.DataFrame(columns=str_map.columns)
for ind, poly in geom_series.iteritems():
    curr_row = str_map.loc[ind[0]].copy(deep=True)
    curr_row['geometry'] = poly
    df = df.append(curr_row)

df_gpd = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'})
df_gpd.to_file("leaflet/SEI.geojson", driver="GeoJSON")

In [ ]:
df_gpd.columns

In [ ]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
# str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = gpd.read_file("MVSEI2014/MVSEI2014.shp")

str_map = fix_crs(str_map)

In [ ]:
str_map.columns

In [ ]:
str_map.Location

In [ ]:
str_map = str_map[['SEI_PolyNb', 'Comp1Lgnd', 'geometry']]

In [ ]:
str_map

In [ ]:
str_map.to_file("leaflet/SEI.geojson", driver="GeoJSON")

In [ ]:
def flatten_gdf_geometry(gdf, geom_type):
    geometry = gdf.geometry
    flattened_geometry = []

    flattened_gdf = gpd.GeoDataFrame()

    for geom in geometry:
        if geom.type in ['GeometryCollection', 'MultiPoint', 'MultiLineString', 'MultiPolygon']:
            for subgeom in geom:
                if subgeom.type==geom_type:
                    flattened_geometry.append(subgeom)
        else:
            if geom.type==geom_type:
                flattened_geometry.append(geom)

    flattened_gdf.geometry=flattened_geometry

    return flattened_gdf

In [ ]:
new_df = flatten_gdf_geometry(str_map, 'Polygon')